In [66]:
import requests
import urllib.parse
from bs4 import BeautifulSoup
import re
import pandas as pd
import time

headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}
# target_url='https://www.linkedin.com/jobs/search?keywords=python&location=Canada&geoId=&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0'

In [67]:
def send_request(target_url,query_params):
    # Make the GET request with the separated URL and query parameters
    response = requests.get(target_url, params=query_params)
    # To check if the request was successful
    if response.status_code == 200:
        # print("Request was successful!")
        time.sleep(15)
        return BeautifulSoup(response.text,'html.parser')
    else:
        print(f"Request failed with status code: {response.status_code}")

In [68]:
#converting string typed count to integer
def job_count_converter(soup):
    count = soup.find(class_='results-context-header__job-count').text
    return  int(re.sub(r'[^0-9]', '', count))  # Remove anything that's not a digit

In [69]:
def crab_job_links_using_keyword(keyword:str):
    # Main body of the URL
    job_search_url = "https://www.linkedin.com/jobs/search"

    # Query parameters as a dictionary, updated with the new keyword
    query_params = {
    "keywords": keyword,
    "location": "Canada",
    "geoId": "",
    "f_TPR": "r604800",
    "position": "1",
    "pageNum": "0",
    "start": "0",
    "sortBy": "DD"
}
    jobs_crabbed  = 0
    jobs_posting_link = [] 
    soup = send_request(job_search_url, query_params)
    number = job_count_converter(soup)
    if number is None:
        number = 2000
    if number > 2000:
        loop_number = 2000
    else:
        loop_number = number
    while jobs_crabbed <= loop_number:
        if not jobs_crabbed == 0:
            query_params["start"] = jobs_crabbed
            print(jobs_crabbed)
            soup = send_request(job_search_url, query_params)
        job_list_html = soup.find('ul', class_="jobs-search__results-list").find_all('li')
        jobs_crabbed += len(job_list_html)
        for job in job_list_html:
            try:
                job_link = job.find('a', class_="base-card__full-link")['href']
            except:
                job_link = None
            jobs_posting_link.append(job_link)
    print(f"{keyword}: {len(jobs_posting_link)}")
    return jobs_posting_link

In [70]:
# import requests
# from bs4 import BeautifulSoup
# import math
# import pandas as pd
# l=[]
# o={}
# k=[]
# headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}
# target_url='https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Python%20%28Programming%20Language%29&location=Las%20Vegas%2C%20Nevada%2C%20United%20States&geoId=100293800&currentJobId=3415227738&start={}'
# for i in range(0,math.ceil(117/25)):

#     res = requests.get(target_url.format(i))
#     soup=BeautifulSoup(res.text,'html.parser')
#     alljobs_on_this_page=soup.find_all("li")
#     print(len(alljobs_on_this_page))
#     for x in range(0,len(alljobs_on_this_page)):
#         jobid = alljobs_on_this_page[x].find("div",{"class":"base-card"}).get('data-entity-urn').split(":")[3]
#         l.append(jobid)

# target_url='https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{}'
# for j in range(0,len(l)):

#     resp = requests.get(target_url.format(l[j]))
#     soup=BeautifulSoup(resp.text,'html.parser')

#     try:
#         o["company"]=soup.find("div",{"class":"top-card-layout__card"}).find("a").find("img").get('alt')
#     except:
#         o["company"]=None

#     try:
#         o["job-title"]=soup.find("div",{"class":"top-card-layout__entity-info"}).find("a").text.strip()
#     except:
#         o["job-title"]=None

#     try:
#         o["level"]=soup.find("ul",{"class":"description__job-criteria-list"}).find("li").text.replace("Seniority level","").strip()
#     except:
#         o["level"]=None



#     k.append(o)
#     o={}

# df = pd.DataFrame(k)
# df.to_csv('linkedinjobs.csv', index=False, encoding='utf-8')
# print(k)

In [71]:
# create a pandas series using list job list posting and save.
def saving_file(jobs_posting_link, keyword:str):
    series = pd.Series(jobs_posting_link)
    series.to_csv(f'{keyword}.csv', index=False)

In [72]:
search_terms_dict = {
    "Technology": "Technology OR IT OR Machine Learning OR Network Security",
    "Healthcare": "Healthcare OR Pharmaceuticals OR Clinical Trials",
    "Finance": "Finance OR Banking OR Retail Banking",
    "Education": "Education OR Online Learning OR Vocational Training",
    "Manufacturing": "Manufacturing OR Aerospace OR Electronics OR Mechanical",
    "Energy": "Energy OR Renewable OR Nuclear Power OR Hydroelectric Power OR Biofuels",
    "Retail": "Retail OR E-commerce OR Online Marketplaces OR Luxury Goods OR Supermarkets",
    "Construction": "Construction OR Real Estate OR Architecture OR Interior Design OR Property Management",
    "Transportation": "Transportation OR Logistics OR Air Freight OR Railroads OR Maritime OR Courier",
    "Hospitality": "Hospitality OR Tourism OR Culinary Arts OR Theme Parks OR Cruise Lines",
    "Professional Services": "Professional Services OR Audit OR Accounting OR Executive Search OR Digital Marketing",
    "Creative Media": "Creative Media OR Video Production OR Digital Content OR Graphic Design OR Music Industry",
    "Telecommunications": "Telecommunications OR Broadband OR Satellite Communications OR ISPs OR VoIP",
    "Agriculture": "Agriculture OR Forestry OR Organic Farming OR Aquaculture OR Agrochemicals",
    "Public Sector": "Public Sector OR Non-Profit OR Civic Organization OR International Development OR Environmental",
    "Science Research": "Science Research OR Nanotechnology OR Genomics OR Environmental Science OR Astrophysics",
    "Engineering": "Engineering OR Robotics OR Biomedical OR Environmental OR Mechanical OR Software",
    "Emerging Industries": "Emerging Industries OR VR OR AR OR CleanTech OR Space Exploration OR FinTech"
}


In [75]:
for keyword in search_terms_dict:
    jobs_posting_link = crab_job_links_using_keyword(keyword)
    saving_file(jobs_posting_link)
    print(f"Done with {keyword} jobs")

58
Request failed with status code: 429


AttributeError: 'NoneType' object has no attribute 'find'